# cmor dev

In [29]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
import xarray as xr
import xclim as xc

import cordex as cx
from cordex import cmor

In [2]:
# cmor.set_options(exit_control="CMOR_EXIT_ON_MAJOR")


def test_cmorizer_mon():
    ds = cx.tutorial.open_dataset("remo_EUR-11_TEMP2_mon")
    eur11 = cx.cordex_domain("EUR-11")
    ds = ds.assign_coords({"lon": eur11.lon, "lat": eur11.lat})
    filename = cmor.cmorize_variable(
        ds,
        "tas",
        mapping_table={"tas": {"varname": "TEMP2"}},
        cmor_table=cx.tables.cmip6_cmor_table("CMIP6_Amon"),
        dataset_table=cx.tables.cordex_cmor_table("CORDEX_remo_example"),
        grids_table=cx.tables.cmip6_cmor_table("CMIP6_grids"),
        CORDEX_domain="EUR-11",
        time_units=None,
        allow_units_convert=True,
    )
    return filename

In [39]:
ds = cx.tutorial.open_dataset("remo_EUR-11_TEMP2_mon")
ds = ds.pint.quantify()
ds

<xarray.Dataset>
Dimensions:                     (time: 12, height2m: 1, rlat: 433, rlon: 433)
Coordinates:
  * rlon                        (rlon) float64 -28.93 -28.82 ... 18.49 18.6
  * rlat                        (rlat) float64 -23.93 -23.82 ... 23.49 23.6
  * height2m                    (height2m) float64 2.0
    lon                         (rlat, rlon) float64 [degrees_north] -10.32 ....
    lat                         (rlat, rlon) float64 [degrees_north] 21.28 .....
  * time                        (time) datetime64[ns] 2000-01-15 ... 2000-12-15
Data variables:
    TEMP2                       (time, height2m, rlat, rlon) float32 [K] 289....
    rotated_latitude_longitude  |S1 b''

In [41]:
ds.pint.dequantify(format="~P")

<xarray.Dataset>
Dimensions:                     (time: 12, height2m: 1, rlat: 433, rlon: 433)
Coordinates:
  * rlon                        (rlon) float64 -28.93 -28.82 ... 18.49 18.6
  * rlat                        (rlat) float64 -23.93 -23.82 ... 23.49 23.6
  * height2m                    (height2m) float64 2.0
    lon                         (rlat, rlon) float64 -10.32 -10.23 ... 68.65
    lat                         (rlat, rlon) float64 21.28 21.32 ... 67.86 67.8
  * time                        (time) datetime64[ns] 2000-01-15 ... 2000-12-15
Data variables:
    TEMP2                       (time, height2m, rlat, rlon) float32 289.4 .....
    rotated_latitude_longitude  |S1 b''

In [15]:
import xclim as xc

# tas_F = xc.units.convert_units_to(tas, "degF")

In [16]:
ds

<xarray.Dataset>
Dimensions:                     (rlon: 433, rlat: 433, height2m: 1, time: 12)
Coordinates:
  * rlon                        (rlon) float64 -28.93 -28.82 ... 18.49 18.6
  * rlat                        (rlat) float64 -23.93 -23.82 ... 23.49 23.6
  * height2m                    (height2m) float64 2.0
    lon                         (rlat, rlon) float64 ...
    lat                         (rlat, rlon) float64 ...
  * time                        (time) datetime64[ns] 2000-01-15 ... 2000-12-15
Data variables:
    TEMP2                       (time, height2m, rlat, rlon) float32 ...
    rotated_latitude_longitude  |S1 ...

In [8]:
tas_F = xc.units.convert_units_to(ds[["TEMP2"]], "degF")
tas_F

NotImplementedError: Source of type `<class 'xarray.core.dataset.Dataset'>` is not supported.

In [9]:
# See the Usage page for details on opening datasets, subsetting and resampling.
ds = xr.tutorial.open_dataset("air_temperature")
tas = (
    ds.air.sel(lat=40, lon=270, method="nearest")
    .resample(time="D")
    .mean(keep_attrs=True)
)
print(tas.attrs["units"])

degK


In [24]:
import pyremo as pr

surflib = pr.data.surflib("EUR-11")

fib = xc.units.convert_units_to(surflib.FIB, "m")
fib

<xarray.DataArray 'FIB' (rlat: 433, rlon: 433)>
[187489 values with dtype=float32]
Coordinates:
  * rlon     (rlon) float64 -28.93 -28.82 -28.71 -28.59 ... 18.38 18.48 18.59
  * rlat     (rlat) float64 -23.93 -23.82 -23.71 -23.59 ... 23.38 23.48 23.59
Attributes:
    grid_mapping:  rotated_pole
    variable:      FIB
    description:   surface geopotential (orography)
    units:         m
    layer:         1.0
    cf_name:       orog
    code:          129

In [27]:
xc.units.convert_units_to(ds.TEMP2, "K")

<xarray.DataArray 'TEMP2' (time: 12, height2m: 1, rlat: 433, rlon: 433)>
array([[[[289.3508 , ..., 295.57257],
         ...,
         [268.58105, ..., 244.87791]]],


       ...,


       [[[290.93027, ..., 296.74765],
         ...,
         [270.2405 , ..., 248.1745 ]]]], dtype=float32)
Coordinates:
  * rlon      (rlon) float64 -28.93 -28.82 -28.71 -28.6 ... 18.37 18.49 18.6
  * rlat      (rlat) float64 -23.93 -23.82 -23.71 -23.6 ... 23.37 23.49 23.6
  * height2m  (height2m) float64 2.0
    lon       (rlat, rlon) float64 ...
    lat       (rlat, rlon) float64 ...
  * time      (time) datetime64[ns] 2000-01-15 2000-02-15 ... 2000-12-15
Attributes:
    long_name:     2m temperature
    units:         K
    code:          167
    leveltype:     105
    grid_mapping:  rotated_latitude_longitude

In [28]:
import cf_xarray.units
import pint_xarray
import xarray as xr

ds = xr.open_dataset(
    "gs://cmip6/CMIP6/CMIP/NCAR/CESM2-FV2/historical/r2i1p1f1/Amon/sfcWind/gn/v20200226/",
    engine="zarr",
)

ds = ds.pint.quantify()


squared_wind = ds["sfcWind"] ** 2

squared_wind.pint.units

<Unit('meter ** 2 / second ** 2')>

In [46]:
import pyremo as pr

surflib = pr.data.surflib("EUR-11")
fib = surflib[["FIB"]].pint.quantify(format="~P")
fib

Magnitude,[[296.7117614746094 285.093017578125 275.188232421875 ... 0.0 0.0 0.0] [309.28656005859375 293.1111145019531 280.2631530761719 ... 0.0 0.0 0.0] [322.68603515625 301.8411865234375 286.03466796875 ... 0.0 0.0 0.0] ... [0.60550457239151 20.039514541625977 240.3848419189453 ... 124.3947982788086 95.12826538085938 110.80191040039062] [0.821212112903595 100.04255676269531 590.6514892578125 ... 85.95282745361328 105.59952545166016 100.37971496582031] [0.4380664825439453 134.84756469726562 630.5753173828125 ... 36.98588180541992 54.492923736572266 48.390995025634766]]
Units,meter


In [47]:
fib.pint.dequantify(format="~P")

<xarray.Dataset>
Dimensions:  (rlat: 433, rlon: 433)
Coordinates:
  * rlon     (rlon) float64 -28.93 -28.82 -28.71 -28.59 ... 18.38 18.48 18.59
  * rlat     (rlat) float64 -23.93 -23.82 -23.71 -23.59 ... 23.38 23.48 23.59
Data variables:
    FIB      (rlat, rlon) float32 296.7 285.1 275.2 267.2 ... 36.99 54.49 48.39
Attributes:
    CDI:          Climate Data Interface version 1.9.8 (https://mpimet.mpg.de...
    Conventions:  CF-1.6
    history:      Wed Jul 01 12:03:07 2020: cdo -f nc copy lib_EUR-11_frac li...
    CDO:          Climate Data Operators version 1.9.8 (https://mpimet.mpg.de...

In [34]:
da = xr.DataArray(name="a", data=[0, 1, 2], dims="x", coords={"x": [1000, 2000, 3000]})

da = da.pint.quantify({"a": "Pa", "x": "m"})
da

Magnitude,[0 1 2]
Units,pascal


In [36]:
da.pint.dequantify()

<xarray.DataArray 'a' (x: 3)>
array([0, 1, 2])
Coordinates:
  * x        (x) int64 1000 2000 3000
Attributes:
    units:    pascal